In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from time import time
from numba import njit
tol = 1e-8
nn = 301
x = 0.0
y = 0.0
betaBX = 200.0
betaFX = 10.5
kappaX = 1.2
kappaY = 0.9
theta = 1.0
alpha = 10.0
betaBY = 400.0
betaFY = 10.0
deltaX = 1.0
deltaY = 0.05
par = betaFX
IC = state = np.array([x,y])

stateDim, parDim = len(state), 1

@njit
def F(t,state,par):  
    x,y = state
    betaFX = par  
    tempX = x * kappaX / theta
    tempY = x * kappaY / theta
    dxdt = betaFX + (betaBX - betaFX) * (tempX**3) / (1 + tempX + tempX**2 + tempX**3) - deltaX*x - alpha*x*y
    dydt = betaFY + (betaBY - betaFY) * (tempY**3) / (1 + tempY + tempY**2 + tempY**3) - deltaY*y - alpha*x*y    
    return np.array([dxdt,dydt])
        
@njit
def J(t,state,par):
    x,y = state
    betaFX = par
    return np.array([[(-kappaX**4*x**3*(betaBX - betaFX)*(3*kappaX**2*x**2 + 2*kappaX*theta*x + theta**2) + 3*kappaX**3*x**2*(betaBX - betaFX)*(kappaX**3*x**3 + kappaX**2*theta*x**2 + kappaX*theta**2*x + theta**3) - (alpha*y + deltaX)*(kappaX**3*x**3 + kappaX**2*theta*x**2 + kappaX*theta**2*x + theta**3)**2)/(kappaX**3*x**3 + kappaX**2*theta*x**2 + kappaX*theta**2*x + theta**3)**2, -alpha*x], [(-alpha*y*(kappaY**3*x**3 + kappaY**2*theta*x**2 + kappaY*theta**2*x + theta**3)**2 - kappaY**4*x**3*(betaBY - betaFY)*(3*kappaY**2*x**2 + 2*kappaY*theta*x + theta**2) + 3*kappaY**3*x**2*(betaBY - betaFY)*(kappaY**3*x**3 + kappaY**2*theta*x**2 + kappaY*theta**2*x + theta**3))/(kappaY**3*x**3 + kappaY**2*theta*x**2 + kappaY*theta**2*x + theta**3)**2, -alpha*x - deltaY]])

@njit
def dFdtheta_constant(t,state,par):
    x,y = state
    betaFX = par
    tempX = x * kappaX / theta    
    return np.array([[1-(tempX**3) / (1 + tempX + tempX**2 + tempX**3)],[0.0]])
 
def jointF(t,jointState,par):
    x = jointState[:stateDim]
    s = jointState[stateDim:].reshape((stateDim,parDim))
    dx = F(t,x,par)
    ds = (J(t,x,par).dot(s)+ dFdtheta_constant(t,x,par)).reshape((stateDim*parDim,))
    return np.hstack((dx,ds))
    
jointState = np.zeros(stateDim*(parDim+1))
jointState[:stateDim] = IC[:]

temp1 = np.zeros(nn)
temp2 = np.zeros(nn)
count = 0
tf_arr = np.linspace(0,20,nn)
for ii in range(1,nn):
    tf = tf_arr[ii]
    tSpan = np.linspace(0,tf,2)    
    count += 1
    print(count)
    for i in range(2):
        tic = time()
        sol_buffer = solve_ivp(fun=lambda t,z: jointF(t,z,par), t_span=(tSpan[0],tSpan[-1]),y0=jointState, t_eval=tSpan, method='LSODA', rtol=tol,atol=tol)
        xx = sol_buffer.y[2,-1]
        temp1[count] = time()-tic
    temp2[count] = xx        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [2]:
np.save('forward1.npy',temp1)
np.save('forward2.npy',temp2)

In [4]:
temp2

array([ 0.00000000e+00,  1.92360324e-01, -5.51209088e-02,  9.57210026e-04,
        1.57743906e-02,  1.68750067e-02,  1.83945974e-02,  1.99768743e-02,
        2.16131643e-02,  2.33057259e-02,  2.50571839e-02,  2.68703376e-02,
        2.87481713e-02,  3.06938699e-02,  3.27108361e-02,  3.48027102e-02,
        3.69733909e-02,  3.92270592e-02,  4.15682052e-02,  4.40016567e-02,
        4.65326130e-02,  4.91666807e-02,  5.19099157e-02,  5.47688696e-02,
        5.77506417e-02,  6.08629383e-02,  6.41141395e-02,  6.75133759e-02,
        7.10706159e-02,  7.47967620e-02,  7.87037704e-02,  8.28047732e-02,
        8.71142387e-02,  9.16481391e-02,  9.64241578e-02,  1.01461922e-01,
        1.06783305e-01,  1.12412701e-01,  1.18377471e-01,  1.24708373e-01,
        1.31440123e-01,  1.38612055e-01,  1.46268928e-01,  1.54461901e-01,
        1.63249734e-01,  1.72700269e-01,  1.82892275e-01,  1.93917762e-01,
        2.05884920e-01,  2.18921880e-01,  2.33181584e-01,  2.48848160e-01,
        2.66145397e-01,  